In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

In [ ]:
dataset_dir = Path('../datasets_secret')
templates_raw = pd.read_csv(dataset_dir / 'templates_w_nodes.csv')


# convert content to json
templates_raw['content_dict'] = templates_raw['content'].apply(json.loads)

templates_raw.head()

templates_raw.columns



In [ ]:
any_text_filter = lambda x: any(node['type'] == 'text' for node in x['nodes'])
any_image_filter = lambda x: any(node['type'] == 'image' for node in x['nodes'])
not_any_geo_filter = lambda x: not any(node['type'] == 'geo' for node in x['nodes'])

# Filter for templates with just images and text, and NEVER geo
templates_images_and_text = templates_raw[
    (templates_raw['content_dict'].apply(any_text_filter)
    | templates_raw['content_dict'].apply(any_image_filter))
    &
    templates_raw['content_dict'].apply(not_any_geo_filter)
]
num_templates = len(templates_images_and_text)
print(f"Number of templates: {num_templates}")

# Print a random template
random_template = templates_images_and_text.iloc[np.random.randint(0, num_templates)]
print(random_template.content)


In [ ]:
# Evaluate UI quality scores for originals and reconstructions
import sys
sys.path.append('..')
from lib.uiclip import score_image
import matplotlib.pyplot as plt

canva_dir = Path('../datasets/canva')
reconstructions_dir = Path('../datasets/reconstructions')

print("Evaluating UI quality scores...")

# Get all original Canva images
canva_images = sorted(canva_dir.glob('*.webp'))

results = []

for img_path in canva_images:
    design_name = img_path.stem
    
    print(f"Scoring {design_name}...")
    
    # Score original
    original_score = score_image(img_path, description="graphic design")
    
    # Score reconstruction if it exists
    reconstruction_path = reconstructions_dir / design_name / "render.png"
    if reconstruction_path.exists():
        reconstruction_score = score_image(reconstruction_path, description="graphic design")
        
        results.append({
            'name': design_name,
            'original': original_score,
            'reconstruction': reconstruction_score,
            'diff': reconstruction_score - original_score
        })

# Convert to DataFrame
df_scores = pd.DataFrame(results)
print(f"\n{df_scores}")

# Plot comparison
if len(df_scores) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Scatter plot: Original vs Reconstruction
    ax1 = axes[0]
    ax1.scatter(df_scores['original'], df_scores['reconstruction'], s=100, alpha=0.6)
    
    # Add diagonal line (perfect reconstruction)
    lims = [
        min(df_scores['original'].min(), df_scores['reconstruction'].min()) - 0.05,
        max(df_scores['original'].max(), df_scores['reconstruction'].max()) + 0.05,
    ]
    ax1.plot(lims, lims, 'k--', alpha=0.3, label='Perfect reconstruction')
    
    ax1.set_xlabel('Original Score', fontsize=12)
    ax1.set_ylabel('Reconstruction Score', fontsize=12)
    ax1.set_title('Original vs Reconstruction Aesthetic Scores', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(lims)
    ax1.set_ylim(lims)
    
    # Bar plot: Difference by design
    ax2 = axes[1]
    colors = ['green' if d >= 0 else 'red' for d in df_scores['diff']]
    ax2.bar(range(len(df_scores)), df_scores['diff'], color=colors, alpha=0.6)
    ax2.axhline(y=0, color='k', linestyle='-', linewidth=0.8)
    ax2.set_xlabel('Design', fontsize=12)
    ax2.set_ylabel('Score Difference (Recon - Original)', fontsize=12)
    ax2.set_title('Aesthetic Score Difference by Design', fontsize=14)
    ax2.set_xticks(range(len(df_scores)))
    ax2.set_xticklabels([name[:15] + '...' if len(name) > 15 else name 
                         for name in df_scores['name']], rotation=45, ha='right')
    ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary stats
    print("\n" + "="*60)
    print("SUMMARY STATISTICS")
    print("="*60)
    print(f"Average Original Score:        {df_scores['original'].mean():.4f}")
    print(f"Average Reconstruction Score:  {df_scores['reconstruction'].mean():.4f}")
    print(f"Average Difference:            {df_scores['diff'].mean():+.4f}")
    print(f"\nBest Reconstruction:  {df_scores.loc[df_scores['reconstruction'].idxmax(), 'name']}")
    print(f"  Score: {df_scores['reconstruction'].max():.4f}")
    print(f"\nWorst Reconstruction: {df_scores.loc[df_scores['reconstruction'].idxmin(), 'name']}")
    print(f"  Score: {df_scores['reconstruction'].min():.4f}")
